In [1]:
# -*- coding: utf-8 -*-
import requests
import time
import random
import json
import tornado.ioloop
import tornado.web
import tornado.httpserver
import tornado.options
import os
import sys
from tornado.options import define, options
import tornado.websocket
import json, ast
import numpy as np
from pymongo import  MongoClient
def get_page(url):
    user_agent_str = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36"
    time.sleep(random.uniform(0,1))
    return json.loads(requests.get(url, headers={"Connection":"keep-alive", "User-Agent": user_agent_str}).text)


def filterData():
    url = "http://123.207.75.151:9999/bch/api/viabtc/txs/get_all/1MEPB525tEHRFLdq6aR8d2t8jaaRQj2iWX"
    dataArr = get_page(url)
    # for i in range(1, 10000):
    #     url = "https://blockchain.info/rawaddr/" + address +'?offset='
    #     url += str(i*50)
    #     data = get_page(url)
    #     try:
    #         if len(data['txs']) == 0:
    #             break
    #         dataArr['txs'].extend(data['txs'])
    #     except: 
    #         break
    return dataArr

In [3]:
data

{'result': {'msg': [{'confirmations': '13054',
    'income': '-1490',
    'inputs': [{'coinbase': '',
      'prev_addresses': ['1MEPB525tEHRFLdq6aR8d2t8jaaRQj2iWX'],
      'prev_txid': '05010d8b254d275c8b746083df2e9594b1505121824cfe47513570f0f684469a',
      'prev_value': '1490'}],
    'is_coinbase': False,
    'outputs': [{'addresses': ['1MEPB525tEHRFLdq6aR8d2t8jaaRQj2iWX'],
      'next_txid': '',
      'value': '489.99999714'},
     {'addresses': ['1G47mSr3oANXMafVrR8UC4pzV7FEAzo3r9'],
      'next_txid': '296e3c698d672bc2cbda481a9d4abb3d88a358f370443e8f85c596d49a248db9',
      'value': '1000'}],
    'position': '0',
    'time': '1524354059',
    'txid': '987cf6cf2ca12fccf36937cc334257f1662be0ff7492cad5cae2dd8a7d5690ed'},
   {'confirmations': '13076',
    'income': '5.27654945',
    'inputs': [{'coinbase': '',
      'prev_addresses': ['1DcKsGnjpD38bfj6RMxz945YwohZUTVLby'],
      'prev_txid': '2ddb6e1feb0204565bcfd4e7746c829468ea1378882769433b012f7730b0c00c',
      'prev_value': '0.054

In [18]:
print(len(data['result']['msg']))

5


In [ ]:
result={}

In [64]:
address='1MEPB525tEHRFLdq6aR8d2t8jaaRQj2iWX'
addrData={}
txData={}
txData['addr']=address
txData['n_tx']=len(data['result']['msg'])
txData['received']=0
txData['sent']=0
txData['balance']=0
txData['txs']=[]
received=0
sent=0
balance=0
for index in data['result']['msg']:
    record={}
    record['inputs']=[]
    record['outputs']=[]
    record['time']=float(index['time'])
    record['confirmations']=index['confirmations']
    record['income']=float(index['income'])
    record['txid']=index['txid']
    balance+=float(index['income'])
    
    for put in index['inputs']:
        temp={}
        temp['txid']=put['prev_txid']
        temp['value']=float(put['prev_value'])
        temp['addr']=put['prev_addresses'][0]
        if temp['addr'] == address:
            sent+=temp['value']
            
        if temp['addr'] not in addrData:
            addrData[temp['addr']]={'addr': temp['addr'], 'sent': temp['value'], 'received': 0, 'balance': -temp['value'], 'input_n': 1, 'output_n': 0, 'tx_n': 1, 'txid': [record['txid']]}
        else:
            addrData[temp['addr']]['sent']+=temp['value']
            addrData[temp['addr']]['balance']-=temp['value']
            addrData[temp['addr']]['input_n']+=1
            addrData[temp['addr']]['tx_n']+=1
            addrData[temp['addr']]['txid'].append(record['txid'])
        record['inputs'].append(temp)
    
 
    for put in index['outputs']:
        temp={}
        temp['txid']=put['next_txid']
        temp['value']=float(put['value'])
        temp['addr']=put['addresses'][0]
        if temp['addr'] == address:
            received+=temp['value']
            
        if temp['addr'] not in addrData:
            addrData[temp['addr']]={'addr': temp['addr'], 'sent': 0, 'received': temp['value'], 'balance': temp['value'], 'input_n': 0, 'output_n': 1, 'tx_n': 1, 'txid': [record['txid']]}
        else:
            addrData[temp['addr']]['received']+=temp['value']
            addrData[temp['addr']]['balance']+=temp['value']
            addrData[temp['addr']]['output_n']+=1
            addrData[temp['addr']]['tx_n']+=1
            addrData[temp['addr']]['txid'].append(record['txid'])
            
        record['outputs'].append(temp)
    #print(record) 
    txData['txs'].append(record)
    


    
txData['received']=received
txData['sent']=sent
txData['balance']=received-sent

temp=[]
for index in addrData:
    temp.append(addrData[index])
print(addrData['1MEPB525tEHRFLdq6aR8d2t8jaaRQj2iWX'])

{'txid': ['987cf6cf2ca12fccf36937cc334257f1662be0ff7492cad5cae2dd8a7d5690ed', '987cf6cf2ca12fccf36937cc334257f1662be0ff7492cad5cae2dd8a7d5690ed', '987cf6cf2ca12fccf36937cc334257f1662be0ff7492cad5cae2dd8a7d5690ed', '987cf6cf2ca12fccf36937cc334257f1662be0ff7492cad5cae2dd8a7d5690ed', 'ac40b543e57a32a6e286f257c22ca37fb6cd0003aaaab72fcc95a60f0838154c', '8eabaa8852c38cfcdf68c39f2965b73d0ab7bc109e4210e175399a58dcb9cc3b', '05010d8b254d275c8b746083df2e9594b1505121824cfe47513570f0f684469a'], 'sent': 2980.0, 'addr': '1MEPB525tEHRFLdq6aR8d2t8jaaRQj2iWX', 'balance': 3545.2765437300004, 'received': 6525.27654373, 'input_n': 2, 'tx_n': 7, 'output_n': 5}
